In [2]:
# check whether a packege is installed 
import importlib.util

package_name = 'lifelines'  # replace with the package you want to check
package_spec = importlib.util.find_spec(package_name)

if package_spec is not None:
    print(f"'{package_name}' is installed.")
else:
    print(f"'{package_name}' is NOT installed.")

'lifelines' is NOT installed.


In [3]:
pip install lifelines

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'autograd-gamma' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'autograd-gamma'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4118 sha256=61d9c8fe6776dd011ef0eb57e16a9a061978105d0c8628c9f81d546a88087dbf
  Stored in directory: /Users/lu/Library/Caches/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [lifelines]/5 [lifelines]
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from lifelines import CoxPHFitter
import numpy as np

In [5]:
# 1. Simulated panel data: one row per loan
np.random.seed(42)
n = 1000
data = pd.DataFrame({
    'loan_age': np.random.randint(1, 60, size=n),          # months since origination
    'credit_score': np.random.normal(700, 40, size=n),
    'loan_amount': np.random.normal(200000, 30000, size=n),
    'rate_diff': np.random.uniform(-0.01, 0.04, size=n),    # current rate - market rate
})


# Simulate prepayment based on higher rate_diff and age
baseline_hazard = 0.005
hazard = baseline_hazard + 0.05 * data['rate_diff'] + 0.0001 * data['loan_age']
prob_event = 1 - np.exp(-hazard * data['loan_age'])  # simplified prepayment probability, assume en exponential survival model
data['prepaid'] = np.random.binomial(1, prob_event)  # mimics realistic event data


# survival model inputs
data['duration'] = data['loan_age']
data['event'] = data['prepaid']  # 1 = prepaid, 0 = still active

In [6]:
# 2. Fit Cox Proportional Hazards model
cph = CoxPHFitter()
cph.fit(data[['duration', 'event', 'credit_score', 'loan_amount', 'rate_diff']], duration_col='duration', event_col='event')

# 3. Output model summary
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 1000 total observations, 764 right-censored observations>
             duration col = 'duration'
                event col = 'event'
      baseline estimation = breslow
   number of observations = 1000
number of events observed = 236
   partial log-likelihood = -1262.20
         time fit was run = 2025-05-27 18:17:28 UTC

---
              coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                     
credit_score  0.00      1.00      0.00           -0.00            0.00                1.00                1.00
loan_amount   0.00      1.00      0.00           -0.00            0.00                1.00                1.00
rate_diff     3.91     49.95      4.62           -5.15           12.97                0.01            4.29e+05

              cmp to    z    p  -log2(p)
covariate                               
credit_score    0.00 0.10 0.92      0.12
loan_amount     0.00 0.61 0.54      0.88
rate_diff       0.00 0.85 0.40      1.33
---
Concordance = 0.54
Partial AIC = 2530.40
log-likelihood ratio test = 1.12 on 3 df
-log2(p) of ll-ratio test = 0.37